In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.cluster import KMeans

# Dataset de partidos
df_partidos = pd.read_csv("partidos_cebollitas.csv")
df_partidos['diferencia_goles'] = df_partidos['goles_local'] - df_partidos['goles_visitante']

X_partidos = df_partidos[['posesion_local (%)', 'tiros_arco_local']]
y_partidos = df_partidos['diferencia_goles']

X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_partidos, y_partidos, test_size=0.2, random_state=42)

# Dataset de jugadores
df_jugadores = pd.read_csv("jugadores_cebollitas.csv")
X_jugadores = df_jugadores[['goles', 'asistencias', 'pases_completados (%)', 'tiros_al_arco']]

In [2]:
pipeline_supervisado = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', Ridge(alpha=1.0))
])
pipeline_supervisado.fit(X_train_p, y_train_p)

Pipeline(steps=[('scaler', StandardScaler()), ('ridge', Ridge())])

In [3]:
pipeline_no_supervisado = Pipeline([
    ('scaler', StandardScaler()),
    ('kmeans', KMeans(n_clusters=3, random_state=42))
])
df_jugadores['cluster'] = pipeline_no_supervisado.fit_predict(X_jugadores)

/Users/joacogalt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [4]:
nuevo_partido = pd.DataFrame({'posesion_local (%)':[58],'tiros_arco_local':[9]})
prediccion_partido = pipeline_supervisado.predict(nuevo_partido)

perfiles_jugadores = df_jugadores[['nombre', 'cluster']].head(10)

print(f"Predicción resultado partido (dif. goles): {prediccion_partido.round(2)}")
print("\nPerfiles jugadores (primeros 10):")
print(perfiles_jugadores)

Predicción resultado partido (dif. goles): [0.25]

Perfiles jugadores (primeros 10):
                nombre  cluster
0         Tara Alvarez        0
1        Carol Mcclain        2
2        Robert Martin        0
3    Mr. Robert Turner        1
4  Christopher Kennedy        1
5  Dr. Justin Anderson        2
6      Sandra Shepherd        0
7          Mark Bowers        2
8         Kenneth Cook        0
9    Christopher Parks        1


In [5]:
import ipywidgets as widgets

posesion_widget = widgets.IntSlider(min=40,max=70,description='Posesión %:')
tiros_widget = widgets.IntSlider(min=1,max=15,description='Tiros al arco:')

def predecir_resultados(posesion, tiros):
    datos = pd.DataFrame({'posesion_local (%)':[posesion],'tiros_arco_local':[tiros]})
    pred = pipeline_supervisado.predict(datos)[0]
    print(f"Diferencia de goles predicha: {pred:.2f}")

widgets.interactive(predecir_resultados, posesion=posesion_widget, tiros=tiros_widget)

interactive(children=(IntSlider(value=40, description='Posesión %:', max=70, min=40), IntSlider(value=1, descr…